In [23]:
import psycopg2
import os
import sqlalchemy
import pandas as pd
import pandas.io.sql as psql
import sys

### convert raw farmpulse data (india table) to dataframe

In [24]:
## connects to postgres database
conn = psycopg2.connect ("dbname = mydatabase user = postgres password = password host = localhost")
cur = conn.cursor()

In [19]:
## generates dataframe by chunks
df = pd.read_sql('select * from call_centre.india', con=conn, chunksize = 1000)

In [11]:
## prints full dataframe in chunks
for chunk_dataframe in df:
    print(chunk_dataframe)

### generate size of farmpulse dataframe

In [21]:
total_size = 0
for chunk_dataframe in df:
    total_size += sys.getsizeof(chunk_dataframe)
    
print(f'total size = {total_size} bytes, {total_size/(1024*1024*1024)} GB')

total size = 18920488084 bytes, 17.621077675372362 GB


### generate small sample (first 100 rows)

In [20]:
sample = pd.read_sql('select * from call_centre.india limit 100', con=conn)
sample

,created_on,state_name,district_name,block_name,latitude,longitude,season,sector,category,crop,query_type,query_text_raw,query_text_translate,query_text_hindi,query_text_english,kcc_answer_raw,kcc_answer_translate,kcc_answer_hindi,kcc_answer_english,id
0,2014-12-03 10:25:20,rajasthan,ganganagar,anupgarh,29.181153,73.199860,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME WEED CONTROL OF BENGAL GRAM,False,None,TELL ME WEED CONTROL OF BENGAL GRAM,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,False,None,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,9883396
1,2014-12-03 10:27:50,rajasthan,chittaurgarh,nimbahera,24.625659,74.681027,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME CONTROL OF BORER IN GRAM ?,False,None,TELL ME CONTROL OF BORER IN GRAM ?,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,False,None,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,9883397
2,2014-12-03 10:28:18,rajasthan,hanumangarh,nohar,29.174841,74.768817,rabi,agriculture,others,others,government schemes,TELL ME SOLAR SBSIDY,False,None,TELL ME SOLAR SBSIDY,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,False,None,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,9883398
3,2014-12-03 10:28:20,rajasthan,bharatpur,kumher,27.342644,77.372638,rabi,agriculture,oilseeds,mustard,plant protection,TELL ME INSECT CONTROL IN MUSTRAD,False,None,TELL ME INSECT CONTROL IN MUSTRAD,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,False,None,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,9883399
4,2014-12-03 10:28:51,rajasthan,nagaur,merta,26.644952,74.030942,rabi,agriculture,cereals,wheat,plant protection,seed treatment of wheat,False,None,seed treatment of wheat,USE CARBENDAZIM 2 GM PER KG SEED,False,None,USE CARBENDAZIM 2 GM PER KG SEED,9883400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2014-12-03 12:57:57,rajasthan,ganganagar,ganganagar,29.903840,73.877190,rabi,agriculture,oilseeds,mustard,fertilizer use and availability,TELL ME SPRAY SOLUBLE NPK,False,None,TELL ME SPRAY SOLUBLE NPK,SPRAY SOLUBLE NPK 19:19:19 GRADE 10 GMLTR WATER,False,None,SPRAY SOLUBLE NPK 19:19:19 GRADE 10 GMLTR WATER,9883490
96,2014-12-03 12:58:47,rajasthan,jodhpur,luni,26.001335,73.002153,rabi,agriculture,oilseeds,"castor (rehri, rendi, arandi)",plant protection,TELL ME FUNGAL DIS. CONTROL IN CASTOR,False,None,TELL ME FUNGAL DIS. CONTROL IN CASTOR,SPRAY OF MANCOZEB 2 GM ACEPHATE 75% SP 2 GM PE...,False,None,SPRAY OF MANCOZEB 2 GM ACEPHATE 75% SP 2 GM PE...,9883491
97,2014-12-03 12:58:56,rajasthan,bharatpur,nadbai,27.214402,77.203149,rabi,agriculture,cereals,wheat,nutrient management,tell me about nutrient mgt. in wheat,False,None,tell me about nutrient mgt. in wheat,"FERTILISER IN WHEAT SOWING TIME 80 KG DAP,70 K...",False,None,"FERTILISER IN WHEAT SOWING TIME 80 KG DAP,70 K...",9883492
98,2014-12-03 13:01:13,rajasthan,bikaner,bikaner,28.022935,73.311916,rabi,agriculture,oilseeds,indian rapeseed and mustard (yellow sarson),plant protection,pest control of mustard,False,None,pest control of mustard,SPRAY OF ACEPHATE 75% SP 2 GM PER LITER WATER,False,None,SPRAY OF ACEPHATE 75% SP 2 GM PER LITER WATER,9883493


In [17]:
sample.to_csv('testsample_farmpulse.csv', sep='\t')

### connect to s3

In [25]:
import boto3
import os

session = boto3.Session(profile_name = 'default')
s3b = session.client('s3', 
        region_name = 'ap-southeast-1')

In [15]:
file_name = 'testsample_farmpulse.csv'
bucket_name = 'sh-farmpulse-stage'
key = f'FarmPulse Cleaned csv (no kcc_raw)/{file_name}'

### upload sample to s3 bucket

In [10]:
local_file = os.path.abspath(file_name)

try:
    s3b.upload_file(local_file, bucket_name, key)
    print("Upload Successful")
except FileNotFoundError:
    print("The file was not found")

Upload Successful


### download sample from s3 bucket

In [26]:
try:
    s3b.download_file(bucket_name, key, file_name)
    print("Download Successful")
except Exception as e:
    print(e)

Download Successful
